# Understanding UMF: Unity Molecular Formula for Ceramic Glazes

This notebook walks through how to calculate the **Unity Molecular Formula (UMF)** for ceramic glazes, step by step.

## What is UMF?

UMF is a way of expressing glaze chemistry where **flux oxide moles are normalized to sum to 1.0**.

The key insight: we're normalizing **moles** (molecular amounts), not weight. This matters because different oxides have very different molecular weights - 100 grams of silica contains far more molecules than 100 grams of lead oxide.

By expressing everything in moles relative to a unity of flux, we can:
- Compare glazes regardless of batch size
- Substitute materials while maintaining chemistry
- Predict glaze behavior from the numbers

## Oxide Roles in Glass Formation

Oxides play different roles in forming a glaze. These categories are guidelines, not rigid rules - some oxides behave differently depending on temperature, concentration, and what else is in the glaze.

### Network Formers
These create the glass structure itself:
- **SiO₂** - The primary glass former. Forms the silica tetrahedra network.
- **P₂O₅** - Glass former, creates opalescence in some glazes.

### Network Modifiers (Fluxes)
These break up the silica network, lowering the melting point:
- **R₂O (alkalis)**: Li₂O, Na₂O, K₂O - Very strong fluxes, high thermal expansion
- **RO (alkaline earths)**: CaO, MgO, BaO, SrO - Moderate fluxes, add durability
- **Other RO**: ZnO, PbO, MnO - Various fluxing behaviors

### Intermediates / Amphoteric
These can act as either network formers or modifiers depending on context:
- **Al₂O₃** - Primary stabilizer. Increases viscosity, prevents running, improves durability.
- **TiO₂** - Can act as network former or modifier. Promotes crystallization.
- **Fe₂O₃** - Colorant, but in high amounts also acts as a flux.

### It's Complicated
- **B₂O₃** - Boron is weird. It forms its own glass network, acts as a strong flux at low temperatures, behaves differently at high temperatures, and its thermal expansion contribution is non-linear. Depending on your glaze and firing temp, you might count it as a flux, as a glass former, or track it separately. There's no single right answer.

### The Unity Convention
In UMF, we choose which oxides to normalize to 1.0. **This choice is yours** - pick the oxides that make sense for understanding YOUR glaze. The traditional approach uses R₂O + RO fluxes, but you might include or exclude oxides based on their actual behavior in your specific glaze.

---
## Step 1: Define Oxide Molecular Weights

First, we need the molecular weight of each oxide. This is calculated from atomic masses.

In [21]:
# Molecular weights in g/mol (from CRC Handbook)
molecular_weight = {
    # Primary Glass Former
    "SiO2": 60.085,
    
    # It's Complicated
    "B2O3": 69.620,
    
    # Other Glass Formers
    "P2O5": 141.945,
    
    # Stabilizer
    "Al2O3": 101.961,
    
    # Amphoteric
    "TiO2": 79.866,
    
    # Alkali Fluxes (R2O)
    "Li2O": 29.881,
    "Na2O": 61.979,
    "K2O": 94.196,
    
    # Alkaline Earth Fluxes (RO)
    "MgO": 40.304,
    "CaO": 56.077,
    "SrO": 103.619,
    "BaO": 153.326,
    
    # Other Fluxes
    "ZnO": 81.379,
    "PbO": 223.199,
    "MnO": 70.937,
    
    # Opacifiers
    "SnO2": 150.709,
    "ZrO2": 123.223,
    
    # Iron (two oxidation states)
    "Fe2O3": 159.688,
    "FeO": 71.844,
    
    # Manganese
    "MnO2": 86.937,
    
    # Copper (two oxidation states)
    "CuO": 79.545,
    "Cu2O": 143.091,
    
    # Other Colorants
    "CoO": 74.932,
    "NiO": 74.692,
    "Cr2O3": 151.990,
    "V2O5": 181.880,
    
    # Bismuth
    "Bi2O3": 465.959,
    
    # Rare Earths
    "CeO2": 172.115,
    "La2O3": 325.809,
    "Nd2O3": 336.482,
    "Pr2O3": 329.814,
    "Er2O3": 382.516,
    "Y2O3": 225.810,
    
    # Other
    "F": 18.998,
}

In [ ]:
from utils import format_umf_table

---
## Step 2: Decide Which Oxides to Include as Fluxes

The fluxes are what we'll normalize to unity. **This is a choice you make** based on what you're trying to understand about your glaze.

### Oxide Characteristics

| Oxide | Character |
|-------|-----------|
| **Li₂O, Na₂O, K₂O** | Strong network modifiers. High thermal expansion. Almost always counted as fluxes. |
| **CaO** | Workhorse flux. Promotes durability and hardness. |
| **MgO** | High-temperature flux. Creates smooth, buttery surfaces. |
| **BaO, SrO** | Similar to CaO. BaO can create unique surfaces. |
| **ZnO** | Promotes crystal growth. Can affect color responses. |
| **PbO** | Historically important low-fire flux. Very strong. Toxic. |
| **MnO** | Flux with colorant properties. Browns/purples. |
| **B₂O₃** | Complicated - see above. Strong flux at low temps. |
| **Fe₂O₃** | Primarily a colorant, but contributes to melt in larger amounts. |
| **CuO, CoO** | Primarily colorants. Some flux character. |
| **Bi₂O₃** | Strong flux. Used in specialty low-fire and lusters. |

The traditional approach counts R₂O + RO oxides. But there's no single "correct" list - choose what makes sense for your glaze and what question you're trying to answer.

In [23]:
my_flux_oxides = [
    "Li2O", "Na2O", "K2O",       # Alkalis
    "CaO", "MgO", "SrO", "BaO",  # Alkaline earths
    "ZnO",                        # Metallic flux
]

---
## Step 3: Example - Calculating UMF from Oxide Percentages

Let's work through a real example. Say we have a glaze with this **fired oxide analysis** (weight %):

In [24]:
# Example glaze oxide analysis (weight %)
oxide_analysis = {
    "SiO2": 67.4,
    "Al2O3": 12.6,
    "K2O": 7.5,
    "CaO": 12.5,
}
oxide_analysis

{'SiO2': 67.4, 'Al2O3': 12.6, 'K2O': 7.5, 'CaO': 12.5}

### Step 3a: Convert Weight % to Moles

Divide each oxide's weight percentage by its molecular weight:

$$\text{moles} = \frac{\text{weight %}}{\text{molecular weight}}$$

In [25]:
moles = {}
for oxide, weight_pct in oxide_analysis.items():
    mw = molecular_weight[oxide]
    moles[oxide] = weight_pct / mw
    print(f"{oxide:8} {weight_pct:6.2f} / {mw:6.2f} = {moles[oxide]:.4f}")

SiO2      67.40 /  60.09 = 1.1217
Al2O3     12.60 / 101.96 = 0.1236
K2O        7.50 /  94.20 = 0.0796
CaO       12.50 /  56.08 = 0.2229


### Step 3b: Sum the Flux Moles

Add up the moles of all flux oxides:

$$\text{flux total} = \sum_{\text{flux oxides}} \text{moles}_i = \text{moles}_{K_2O} + \text{moles}_{CaO} + \cdots$$

This sum becomes our divisor - what we normalize everything against.

In [26]:
flux_total = sum(moles[ox] for ox in moles if ox in my_flux_oxides)
flux_total

0.30252900284469525

### Step 3c: Divide by Flux Total to Get UMF

Divide **every** oxide's moles by the flux total:

$$\text{UMF}_{\text{oxide}} = \frac{\text{moles}_{\text{oxide}}}{\text{flux total}}$$

For example, with our values:

$$\text{UMF}_{SiO_2} = \frac{1.122}{0.303} = 3.70$$

$$\text{UMF}_{K_2O} = \frac{0.080}{0.303} = 0.26$$

Now the fluxes sum to 1.0, and everything else is expressed relative to that unity.

In [27]:
# Calculate UMF
umf = {}
for oxide, mol in moles.items():
    umf[oxide] = mol / flux_total

# Display using format helper
flux_list = [ox for ox in ["K2O", "CaO"] if ox in umf]
print(format_umf_table(umf, flux_oxides=flux_list))

Flux:
  K2O      0.263
  CaO      0.737
  TOTAL    1.000

Other:
  SiO2     3.708
  Al2O3    0.408


---
## Interpreting the UMF

Now we can read the glaze chemistry. The UMF tells us about molecular proportions, which relate to how the glaze behaves.

### Reading the Fluxes

The flux unity (1.0) is split among your flux oxides:
- **K₂O: 0.26** + **CaO: 0.74** = 1.0

This tells us it's a **calcium-dominant** glaze. Calcium promotes durability and hardness. If it were sodium-dominant, we'd expect higher thermal expansion and more fluidity.

The **alkali:alkaline earth ratio** matters:
- More alkali (Li, Na, K) → more fluid, higher expansion, brighter colors
- More alkaline earth (Ca, Mg, Ba, Sr) → more stable, more durable

### Reading Al₂O₃

**Al₂O₃: 0.41**

Alumina is the stabilizer. It:
- Increases viscosity (prevents running)
- Improves durability and chemical resistance
- Affects matteness (more Al₂O₃ → more matte)

Typical ranges:
- 0.2-0.4 for glossy glazes
- 0.4-0.6 for satin/matte

### Reading SiO₂

**SiO₂: 3.70**

Silica is the glass former. More silica means:
- Higher melting temperature
- Harder, more durable surface
- More resistance to acids

### The SiO₂:Al₂O₃ Ratio

$$\text{Si:Al ratio} = \frac{\text{UMF}_{SiO_2}}{\text{UMF}_{Al_2O_3}} = \frac{3.70}{0.41} = 9.0$$

This ratio strongly predicts surface quality:

| Ratio | Surface | Character |
|-------|---------|-----------|
| 3-5 | Matte | High alumina stiffens the melt |
| 6-8 | Satin | Balanced |
| 9+ | Glossy | Silica-rich, fluid melt |

A ratio of 9.0 suggests a glossy surface.

In [28]:
sio2_al2o3 = umf["SiO2"] / umf["Al2O3"]
sio2_al2o3

9.077314280959119

---
## Summary: The UMF Calculation

1. **Start with oxide weight percentages** (from fired analysis or recipe calculation)

2. **Convert to moles**: `moles = weight% / molecular_weight`

3. **Sum flux moles**: Add up all R₂O and RO oxides

4. **Divide all oxides by flux sum**: `UMF_value = moles / flux_sum`

Result: Fluxes sum to 1.0, everything else is expressed relative to that unity.

### Why This Matters

- **Comparison**: Two glazes with identical UMF but different materials should behave similarly
- **Substitution**: Know what you're changing when you swap materials
- **Prediction**: Ratios like SiO₂:Al₂O₃ predict surface quality